In [ ]:
import sys, tweepy
from twitter_api_credentials import twitter_api_credentials
import pandas as pd

userID = 'elonmusk'

#### Testing API Pull

In [ ]:
auth = tweepy.OAuthHandler(twitter_api_credentials['API Key'],
                           twitter_api_credentials['API Key Secret'])
auth.set_access_token(twitter_api_credentials['Access Token'],
                      twitter_api_credentials['Access Token Secret'])
api = tweepy.API(auth)

#tweets = api.user_timeline(screen_name=userID, 
#                          count=200,
#                           include_rts=False,
#                           tweet_mode = 'extended')

In [ ]:
# Test API
#for info in tweets[:3]:
#  print('ID: {}'.format(info.id))
#  print(info.created_at)
#  print(info.full_text)
#  print('\n')

ID: 1463220220504518665
2021-11-23 18:57:40
@WholeMarsBlog Exactly (sigh)


ID: 1463200622673551366
2021-11-23 17:39:48
@cz_binance @CoinDesk @binance @egreechee Sounds good


ID: 1463199222895988742
2021-11-23 17:34:14
@flcnhvy @WholeMarsBlog It can simulate anywhere, but we need to focus on one region until FSD is out of beta, then expand geographically to have betas in other countries. Canada is not a lot different from US, but different enough.




#### Actual Pull Code

Note: Get the oldest id from the last row in the csv file; this API pull will continue to pull tweets beyond the last tweet captured

In [ ]:
all_tweets = []
#all_tweets.extend(tweets)
#first_needed_id = 1462905842391560000
#oldest_id = 1327726256880710000 #<-- Accurate; need to let API reload
# use the following the re-retrieve twitter data
oldest_id = 1331310252927676416


while True:
  tweets = api.user_timeline(screen_name=userID,
                             count=200,
                             include_rts=False,
                             max_id=oldest_id - 1,
                             tweet_mode='extended')
  
  if len(tweets) == 0:
    print(f'Last ID Received: oldest_id')
    break
  oldest_id = tweets[-1].id
  all_tweets.extend(tweets)
  print('N of tweets downloaded till now {}'.format(len(all_tweets)))

Last ID Received: oldest_id


In [ ]:
oldest_id

1384579374607831042

In [ ]:
print(len(all_tweets))

0


Save results to csv

In [ ]:
# Unpack tweet items
# Note: need to store ID's as type 'object' => will get rounded otherwise
outtweets = [[tweet.id_str,
              tweet.created_at,
              tweet.favorite_count,
              tweet.retweet_count,
              tweet.full_text.encode('utf-8').decode('utf-8')]
             for idx, tweet in enumerate(all_tweets)]

# Extract image urls
image_urls = []

for status in all_tweets:
  media = status.entities.get('media', [])

  if(len(media) > 0):
    image_urls.append(media[0]['media_url'])
  else:
    image_urls.append('')

In [ ]:
df = pd.DataFrame(outtweets,
                  columns=['id',
                           'created_at',
                           'favorite_count',
                           'retweet_count',
                           'full_text'])

# Add image urls as attribute
df['image_urls'] = image_urls

df.to_csv('elon_musk_twitter_data3.csv',
          mode='a',
          index=False,
          encoding='utf-8')

df.head()

id  ... image_urls
0  1393993975019749382  ...           
1  1393991411524120576  ...           
2  1393983247311380482  ...           
3  1393979913624072194  ...           
4  1393976234477072395  ...           

[5 rows x 6 columns]

In [ ]:
df.loc[df.image_urls != '', ['image_urls']].tail()

image_urls
732  http://pbs.twimg.com/media/E-NnYErVUAINQTs.jpg
735  http://pbs.twimg.com/media/E-JkZaKVIAcbTdW.jpg
743  http://pbs.twimg.com/media/E-FBJGiX0A47aNS.jpg
744  http://pbs.twimg.com/media/E-E92VyWQAI_DSc.jpg
745  http://pbs.twimg.com/media/E-E7PSMWYAsbU5D.jpg

In [ ]:
import urllib.request

urllib.request.urlretrieve('http://pbs.twimg.com/media/FEsM1ejWYAI3px1.jpg', 'test_image.jpg')

('test_image.jpg', <http.client.HTTPMessage at 0x7f5b84798890>)

### References

https://fairyonice.github.io/extract-someones-tweet-using-tweepy.html

Media Images:<br>
https://miguelmalvarez.com/2015/03/03/download-the-pictures-from-a-twitter-feed-using-python/
